# Training Phase-1 (Simple CNN)

In [1]:
# Install required packages
from distutils.dir_util import copy_tree
copy_tree("C://Users//aksha//Desktop//Term 3//project//Project_work//Capstone-project-aml//lib", "..//Capstone Project//Working//")
!pip install efficientnet_pytorch


import os
import csv
from collections import defaultdict
import sys
sys.path.append("C://Users//aksha//Desktop//Term 3//project//Project_work//Capstone-project-aml//results//results//lib")

import torch.optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import OneCycleLR

In [2]:
from training import load_classifier_transforms, cycle, save_state

In [3]:
from models import CLASSIFIER_MODEL_GENERATORS

In [4]:
from data import CLASSES
from datasets import OrthonetClassificationDataset

In [5]:
# Train only 1 model architecture
STOP_AFTER_EFFICIENTNET = True

In [6]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

CSV_TRAIN_VAL = "C://Users//aksha//Desktop//Term 3//project//Project_work//Capstone-project-aml//data//archive//train.csv"
DATA_PATH = "C://Users//aksha//Desktop//Term 3//project//Project_work//Capstone-project-aml//data//archive//orthonet data//orthonet data new"
MODEL_DIR = "C://Users//aksha//Desktop//Term 3//project//Project_work//Capstone Project//Working//"

WEIGHT_LOSS = True
BS_TRAIN = 32
BS_VAL = 32
N_WORKERS = 2
N_EPOCHS = 300
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 5e-4

In [7]:
ds_train = OrthonetClassificationDataset('train', CSV_TRAIN_VAL, DATA_PATH, None)
ds_val = OrthonetClassificationDataset('val', CSV_TRAIN_VAL, DATA_PATH, None)

print(f"TRAIN")
ds_train.stats()
print(f"VAL")
ds_val.stats()

Found 918 train samples from C://Users//aksha//Desktop//Term 3//project//Project_work//Capstone-project-aml//data//archive//train.csv

Found 251 val samples from C://Users//aksha//Desktop//Term 3//project//Project_work//Capstone-project-aml//data//archive//train.csv

TRAIN
396 unique patients
Class                                             Number of samples
Hip_SmithAndNephew_Polarstem_NilCol               51
Knee_SmithAndNephew_GenesisII                     117
Hip_Stryker_Exeter                                192
Knee_Depuy_Synthes_Sigma                          78
Hip_DepuySynthes_Corail_Collar                    102
Hip_DepuySynthes_Corail_NilCol                    128
Hip_JRIOrtho_FurlongEvolution_Collar              22
Knee_SmithAndNephew_Legion2                       29
Hip_Stryker_AccoladeII                            34
Hip_SmithAndNephew_Anthology                      88
Hip_JRIOrtho_FurlongEvolution_NilCol              22
Knee_ZimmerBiomet_Oxford                          5

In [ ]:







results_by_model_by_epoch = defaultdict(lambda: defaultdict(list))

for model_type, model_generator in CLASSIFIER_MODEL_GENERATORS.items():

    # Data
    train_transforms, test_transforms = load_classifier_transforms()
    ds_train = OrthonetClassificationDataset('train', CSV_TRAIN_VAL, DATA_PATH, train_transforms)
    ds_val = OrthonetClassificationDataset('val', CSV_TRAIN_VAL, DATA_PATH, test_transforms)
    dl_train = DataLoader(ds_train, BS_TRAIN, shuffle=True, num_workers=N_WORKERS, pin_memory=True)
    dl_val = DataLoader(ds_val, BS_VAL, shuffle=True, num_workers=N_WORKERS, pin_memory=True)

    # Model
    model = model_generator(n_in=1, n_out=len(CLASSES)).to(DEVICE)
    optimizer = torch.optim.AdamW((p for p in model.parameters() if p.requires_grad), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = OneCycleLR(optimizer, max_lr=LEARNING_RATE*10, steps_per_epoch=len(dl_train), epochs=N_EPOCHS)
    train_criterion = nn.CrossEntropyLoss(weight=ds_train.get_class_weights().to(DEVICE) if WEIGHT_LOSS else None)
    test_criterion = nn.CrossEntropyLoss(weight=ds_train.get_class_weights().to(DEVICE) if WEIGHT_LOSS else None)

    # Train
    best_loss, best_path, last_save_path = 1e10, None, None

    print(f"Training {model_type}")
    for epoch in range(1, N_EPOCHS + 1):
        train_loss, train_acc = cycle('train', model, dl_train, DEVICE, epoch, train_criterion, optimizer, scheduler)
        val_loss, val_acc = cycle('test', model, dl_val, DEVICE, epoch, test_criterion, optimizer)

        print(f"Epoch {epoch:03d}\t\tTRAIN loss: {train_loss:.4f}\tTRAIN acc: {train_acc:.4f}\tVAL loss: {val_loss:.4f}{'*' if val_loss < best_loss else ''}\tVAL acc: {val_acc:.4f}")

        state = {'epoch': epoch + 1,
                 'state_dict': model.state_dict(),
                 'optimizer': optimizer.state_dict(),
                 'scheduler': scheduler}
        save_path = os.path.join(MODEL_DIR, f"{model_type}_{epoch}_{val_loss:.07f}.pt")
        best_loss, last_save_path = save_state(state, save_path, val_loss, best_loss, last_save_path)

        results_by_model_by_epoch[model_type]['train_loss'].append(train_loss)
        results_by_model_by_epoch[model_type]['train_acc'].append(train_acc)
        results_by_model_by_epoch[model_type]['val_loss'].append(val_loss)
        results_by_model_by_epoch[model_type]['val_acc'].append(val_acc)

    with open(os.path.join(MODEL_DIR, f"{model_type}_{best_loss}.txt"), 'w') as f:
        writer = csv.writer(f)
        writer.writerow(results_by_model_by_epoch[model_type].keys())
        writer.writerows(zip(*results_by_model_by_epoch[model_type].values()))
        
    if STOP_AFTER_EFFICIENTNET:
        break

Found 918 train samples from C://Users//aksha//Desktop//Term 3//project//Project_work//Capstone-project-aml//data//archive//train.csv

Found 251 val samples from C://Users//aksha//Desktop//Term 3//project//Project_work//Capstone-project-aml//data//archive//train.csv

Training efficientnet
Epoch 001		TRAIN loss: 2.5215	TRAIN acc: 0.0505	VAL loss: 2.4876*	VAL acc: 0.0000
Epoch 002		TRAIN loss: 2.4864	TRAIN acc: 0.1174	VAL loss: 2.4884	VAL acc: 0.1474
Epoch 003		TRAIN loss: 2.4973	TRAIN acc: 0.1524	VAL loss: 2.4873*	VAL acc: 0.1460
Epoch 004		TRAIN loss: 2.4664	TRAIN acc: 0.1755	VAL loss: 2.4889	VAL acc: 0.1023
Epoch 005		TRAIN loss: 2.4638	TRAIN acc: 0.1646	VAL loss: 2.4924	VAL acc: 0.1023
Epoch 006		TRAIN loss: 2.4432	TRAIN acc: 0.1894	VAL loss: 2.4946	VAL acc: 0.1045
Epoch 007		TRAIN loss: 2.4436	TRAIN acc: 0.2006	VAL loss: 2.4993	VAL acc: 0.1045
Epoch 008		TRAIN loss: 2.4370	TRAIN acc: 0.1949	VAL loss: 2.4953	VAL acc: 0.1045
Epoch 009		TRAIN loss: 2.4635	TRAIN acc: 0.1691	VAL loss: 2.